In [1]:
from scipy.optimize import minimize
import numpy as np

## Задание 1. 
Решить аналитически и проверить при помощи оптимизатора в Python. Оптимизатор можно использовать на своё усмотрение.

In [3]:
def objective(x):
    return np.sum(x**2)

# Определение ограничения
def constraint_eq(x):
    return np.sum(x**4) - 1

x0 = np.full(10, 0**(1/4))
opt_method = 'SLSQP'
res = minimize(objective, x0, method=opt_method, constraints={'type': 'eq', 'fun': constraint_eq})

print("Минимальное значение функции:", res.fun)
print("Точка минимума:", res.x)

Минимальное значение функции: 0.0
Точка минимума: [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


In [4]:
initial_guess = np.array([10**(1/4)]*10)

method = 'SLSQP'

result = minimize(objective, initial_guess, method=method, constraints={'type': 'eq', 'fun': constraint_eq})

print("Минимальное значение функции:", result.fun)
print("Точка минимума:", result.x)

Минимальное значение функции: 3.1622776601542704
Точка минимума: [0.56234425 0.56234425 0.56234425 0.56234425 0.56234042 0.56234042
 0.56234039 0.56234039 0.5623427  0.56233191]


#### Заметно, что аналитическое решение было корректным

## Задача 2.
1. Решить аналитически и проверить при помощи оптимизатора в Python. Оптимизатор можно использовать на своё усмотрение (например, ORTools).
2. Также дополнительно помимо оптимизатора использовать какой-нибудь метаэвристический алгоритм (имитация отжига / квантовый отжиг / муравьиный алгоритм / генетический алгоритм) для проверки результатов.
3. Дать оценку устойчивости метаэвристики в зависимости от начальной точки и от количества итераций.

In [7]:
import random
import numpy as np
from numpy.random import choice
from ortools.constraint_solver import routing_enums_pb2, pywrapcp

class ACO_Solver:
    def __init__(self, cost_matrix, ants, elite_ants, iterations, evaporation, alpha=1, beta=1):
        self.cost_matrix = cost_matrix
        self.pheromones = np.ones_like(cost_matrix) / len(cost_matrix)
        self.ants = ants
        self.elite_ants = elite_ants
        self.iterations = iterations
        self.evaporation = evaporation
        self.alpha = alpha
        self.beta = beta
        self.total_nodes = len(cost_matrix)
        self.nodes_list = range(self.total_nodes)

    def optimize(self):
        best_route = None
        best_length = float("inf")
        for _ in range(self.iterations):
            generated_routes = self.construct_routes()
            self.enhance_pheromones(generated_routes)
            current_best = min(generated_routes, key=lambda x: x[1])
            if current_best[1] < best_length:
                best_route, best_length = current_best
            self.pheromones *= self.evaporation
        return best_route, best_length

    def construct_routes(self):
        routes = []
        for _ in range(self.ants):
            tour = self.generate_tour(0)
            routes.append((tour, self.evaluate_route(tour)))
        return routes

    def generate_tour(self, start):
        path = []
        visited = {start}
        prev_node = start
        for _ in range(self.total_nodes - 1):
            next_node = self.pick_next_node(prev_node, visited)
            path.append((prev_node, next_node))
            prev_node = next_node
            visited.add(next_node)
        path.append((prev_node, start))
        return path

    def pick_next_node(self, current, visited):
        local_pheromones = np.copy(self.pheromones[current])
        local_pheromones[list(visited)] = 0
        weights = (local_pheromones ** self.alpha) * (1.0 / self.cost_matrix[current]) ** self.beta
        weights /= weights.sum()
        return choice(self.nodes_list, 1, p=weights)[0]

    def evaluate_route(self, path):
        return sum(self.cost_matrix[i][j] for i, j in path)

    def enhance_pheromones(self, routes):
        top_routes = sorted(routes, key=lambda x: x[1])[:self.elite_ants]
        for route, length in top_routes:
            for i, j in route:
                self.pheromones[i][j] += 1.0 / length


def setup_problem():
    params = {}
    params["cost_matrix"] = [
        [0, 4, 5, 7, 5],
        [8, 0, 5, 6, 6],
        [3, 5, 0, 9, 6],
        [3, 5, 6, 0, 2],
        [6, 2, 3, 8, 0],
    ]
    params["vehicles"] = 1
    params["start"] = 0
    return params


def display_result(params, manager, model, solution):
    print(f"Optimal cost: {solution.ObjectiveValue()}")
    for vid in range(params["vehicles"]):
        idx = model.Start(vid)
        route_plan = f"Vehicle {vid} follows route:\n"
        while not model.IsEnd(idx):
            route_plan += f" {manager.IndexToNode(idx) + 1} ->"
            idx = solution.Value(model.NextVar(idx))
        route_plan += f" {manager.IndexToNode(idx) + 1}\n"
        print(route_plan)


def solve_vrp():
    params = setup_problem()
    manager = pywrapcp.RoutingIndexManager(
        len(params["cost_matrix"]), params["vehicles"], params["start"]
    )
    model = pywrapcp.RoutingModel(manager)

    def distance_function(i, j):
        return params["cost_matrix"][manager.IndexToNode(i)][manager.IndexToNode(j)]

    transit_callback_idx = model.RegisterTransitCallback(distance_function)
    model.SetArcCostEvaluatorOfAllVehicles(transit_callback_idx)
    model.AddDimension(transit_callback_idx, 0, 3000, True, "Distance")
    search_config = pywrapcp.DefaultRoutingSearchParameters()
    search_config.first_solution_strategy = routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC
    solution = model.SolveWithParameters(search_config)
    if solution:
        display_result(params, manager, model, solution)


if __name__ == "__main__":
    solve_vrp()

Optimal cost: 18
Vehicle 0 follows route:
 1 -> 2 -> 4 -> 5 -> 3 -> 1



Результаты совпали с аналитическим решением, что подтверждает корректность метода.

Стоимость маршрута зависит от начальной температуры: чем она ближе к минимальному значению, тем выше вероятность получения некорректного решения.